**Software Design Decisions**

Use this to document particular decisions and philosophy about the functions produced each week. This should help me with the final 1250 word report.

*Week 14*

Use the python format to force types of inputs and outputs, and use ''' comment  ''' to describe the function so that it shows up on pylance when we hover over it

In [ ]:
def euler_step(f, x: np.ndarray, t: float, h: float) -> np.ndarray:
    '''
    ODE is x' = f(x,t)
    Performs single Euler step with step size h
    '''
    x_next = x + h*f(x,t)
    return x_next

Raise exceptions if the inputs are faulty:

In [ ]:
if deltat_max >= (t_final - t_init):
        raise Exception("Maximum time-step deltat_max >= total time interval")

For euler (and fixed step size) method, use as many steps of size h as we can until we can't, then finish with smaller value of h. Do this with `np.arange(t_init, t_final - h/100, h)` . Not sure this works, might have to check the logic. 

Store the state values as rows in a np array, and at each update use np.vstack to update. Seperately have t_vals as its own 1d array, and then finally `store = np.concatenate((np.array([t_vals]).T, x_store), axis=1)` which will give an output of column 0 with time values, then the remaining columns the state values for each variable, each row a new time.

Within runge-kutta method, explicitly wrote out code to compute each step, but could make a function that computes a single runge kutta step for more modular code.

Spent a lot of time making exercise ipynb for week 14 look good, but don't need to do this for future weeks as this isnt the final product we're assessed on, instead its the functions in the repository plus a demo. But can still copy the formatting for clarity. Plotted things for each exercise very neatly, can steal plotting.

*Week 15*  

We use scipy.optimize.root to find roots of the limit cycle conditions G, phi. We use this because it is robust, well documented and supported, although with time we could perhaps make our own basic newton method, though this might not work well for a variety of badly behaved cases

We make a limit cycle condition function, which we need to find the roots for, in order to find a periodic orbit. This is formatted for use with scipy, by packing the params u0, T into an array.
We use the derivative phase condition, since this requires less user input than the constant phase condition. 

For most smooth functions the orbit should always have a turning point in all variables at some point along the orbit, so this measure is more robust. The phase condition looks for a point along a solution with 0 derivative in the first state variable, which returns to the same point after a certain amount of time. 
This algorithm might inadvertently identify equillibrium solutions, since this behaviour is a subset of equillibrium behaviour. Implementing a constant phase condition might avoid this, since we can specify a point x=a along the orbit which the orbit must start from. So we also code for a constant phase condition, and provide an option for both

We use typing.Literal to give hints for method types in a function call, which shows up when you highlight it in pylance

We make the decision to always put arguments in the order: (Time, State Variables), so that we can extract the time easily and the rest are then state variables

We make an extensive function description, need to go back at the end and format all functions with the same description, need to be usable by someone that has forgotten the function's use. Need to also at the end go over naming scheme and make consistent between objects for ease of use

We use lambda functions to set the values of certain parameters before passing the limit_cycle_condition into the scipy root finder. These can be specified as arguments in our more
general limit cycle finding function


Need to change function descriptions and have a consistent philosophy between them - does a function description refer to the descriptions for other functions or should they be at risk of repeating info found in another. if its a function that isnt for use by the user then does it need to be well commented


After testing on non autonomous forced damped SHM oscillator, we find that checking against the next loop isn't enough - our code found 2 loops that then decayed to another stable limit cycle. We implement additional functionality within the limit cycle condition to check that the first N loops match up with eachother, which passes on to scipi root which minimises the error between the start position and the positions after the first N periods. This increases the time our code takes.
This also means that we can't use the 'hybr' method used by scipi root, as our output [G_collection, phi] has size 2*N+1, whereas our inputs [T, u0] has size 3, and 'hybr' requires these to have the same size! We change the root finding method to lm 

Week 16

We set a certain formatting style for function docstrings, found in week15funcs
This is using / inspired by numpy docstrings

Use https://numpydoc.readthedocs.io/en/latest/format.html 

Need to keep all functions in one location so that they can access eachother

We edit the solve_to function, making new rk4_step and rk2_step functions,
and putting the 'if' statements seperating the methods further on, in line with DRY so that the repeated code is shared

We find scientific problem where the root solver might try to find an equillibrium solution as a limit cycle. The root solver used then tried values of the period lower than our step size deltat_max, causing a consistent error to be called. There were no root solvers with bounds readily available on scipy, so we opt to reframe the problem by instead minimizing the squares of G_collection and phi, and using scipy's minimization function which can take in bounds. We then put bounds on the periods searched, which removed this problem. 

We also specify that any period found must be greater than 10*deltatmax to stop the solver from thinking steps very near eachother are periodic because the state variables haven't changed much

We implement a fudge factor *10 to the objective function to make sure that the minimizer actually obeys the phase condition

We add the period to the objective function: we want the smallest period possible that works, and not a multiple of it, so we add L1 regularisation with a factor alpha

We go through and make really basic error messages for the range of valid inputs and relations between inputs, such as making sure t_final > t_init, and that deltat_max>0. We then only include these error messages in the functions they will be used in, and so any function then just calling more basic functions will get an error message



Week 17: 
Continuation 

Week 19:

Followed lectures for most maths content

Procedurally generate the b vector and A matrix by making use of zeros, eye to make a general dirichlet model, then adding things for other boundary conditions. Using dense matrices for ease of use, but could go through if necessary to produce massive speedups via sparse linear algebra.

We follow the same procedure in the lectures to do it for the left hand sound boundary for each, then access the ends regardless of the matrix A's size

We use linalg.solve to solve things, which is O(N^2): if we have time we could implement the Thomas algorithm O(N). This is used whenever the source term q is linear.

If q is nonlinear and dependent on the solution u, then we have solve with scipy root finding, by introducing an objective function. This requires a guess solution, which we generate by basic results between dirichlet boundary points when we have them

We introduce an idea of the 'interior' of grid points used in actual solving, ie the grid points not included in any dirichlet boundary. Then we can do all operations on these, and depending on grid points add dirichlet points at the end, which are concatenated

In our solver function we specify the number of gridpoints, and boundary conditions.
For the source q we request that the user specifies if it is nonlinear or not, since this produces the biggest dividing line in our code. We also give the user an option to specify a guess solution for use when it is nonlinear
